In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    if b > a:
        b = a^b
        a = a^b
        b = a^b
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['980-2  ', '4-1    ', '62-21  ', '75-35  ', '73-2   '] ['978 ', '3   ', '41  ', '40  ', '71  ']


In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False  True False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False  True False False False False False False
   False]]

 [[False False False False False False False False  True False False
   False]
  [False False False False False False False False False False False
    True]
  [False False False False False False False False  True False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False  True False False False False False
   False]
  [False False False False False False Fal

In [12]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [13]:
for iteration in range(10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=10,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Train on 18000 samples, validate on 2000 samples
Epoch 1/10
18000/18000 [==============================] - 5s 262us/step - loss: 1.8885 - acc: 0.3246 - val_loss: 1.7422 - val_acc: 0.3422
Epoch 2/10
18000/18000 [==============================] - 3s 183us/step - loss: 1.7249 - acc: 0.3528 - val_loss: 1.7130 - val_acc: 0.3574
Epoch 3/10
18000/18000 [==============================] - 3s 183us/step - loss: 1.6662 - acc: 0.3740 - val_loss: 1.6331 - val_acc: 0.3832
Epoch 4/10
18000/18000 [==============================] - 4s 216us/step - loss: 1.6036 - acc: 0.3987 - val_loss: 1.5768 - val_acc: 0.4114
Epoch 5/10
18000/18000 [==============================] - 5s 255us/step - loss: 1.5457 - acc: 0.4251 - val_loss: 1.5227 - val_acc: 0.4276
Epoch 6/10
18000/18000 [==============================] - 4s 233us/step - loss: 1.4828 - acc: 0.4508 - val_loss: 1.4637 - val_acc: 0.4591
Epoch 7/10


Epoch 5/10
18000/18000 [==============================] - 5s 262us/step - loss: 0.4302 - acc: 0.8560 - val_loss: 0.4792 - val_acc: 0.8226
Epoch 6/10
18000/18000 [==============================] - 4s 232us/step - loss: 0.4150 - acc: 0.8596 - val_loss: 0.4524 - val_acc: 0.8371
Epoch 7/10
18000/18000 [==============================] - 4s 218us/step - loss: 0.3864 - acc: 0.8739 - val_loss: 0.4275 - val_acc: 0.8457
Epoch 8/10
18000/18000 [==============================] - 4s 226us/step - loss: 0.3623 - acc: 0.8856 - val_loss: 0.4088 - val_acc: 0.8586
Epoch 9/10
18000/18000 [==============================] - 4s 231us/step - loss: 0.3375 - acc: 0.8968 - val_loss: 0.3786 - val_acc: 0.8734
Epoch 10/10
18000/18000 [==============================] - 4s 210us/step - loss: 0.3197 - acc: 0.9042 - val_loss: 0.3852 - val_acc: 0.8608
Q 933-933 T 0    ☒ 10  
Q 951-592 T 359  ☒ 360 
Q 387-78  T 309  ☒ 300 
Q 879-76  T 803  ☒ 802 
Q 509-70  T 439  ☒ 449 
Q 864-46  T 818  ☑ 818 
Q 227-38  T 189  ☒ 188 
Q 5

18000/18000 [==============================] - 4s 239us/step - loss: 0.0547 - acc: 0.9890 - val_loss: 0.1239 - val_acc: 0.9560
Epoch 2/10
18000/18000 [==============================] - 5s 256us/step - loss: 0.0545 - acc: 0.9888 - val_loss: 0.1157 - val_acc: 0.9581
Epoch 3/10
18000/18000 [==============================] - 4s 218us/step - loss: 0.0929 - acc: 0.9718 - val_loss: 0.1852 - val_acc: 0.9316
Epoch 4/10
18000/18000 [==============================] - 4s 239us/step - loss: 0.0736 - acc: 0.9799 - val_loss: 0.1113 - val_acc: 0.9634
Epoch 5/10
18000/18000 [==============================] - 4s 244us/step - loss: 0.0445 - acc: 0.9925 - val_loss: 0.1012 - val_acc: 0.9642
Epoch 6/10
18000/18000 [==============================] - 4s 233us/step - loss: 0.0414 - acc: 0.9932 - val_loss: 0.1102 - val_acc: 0.9599
Epoch 7/10
18000/18000 [==============================] - 4s 234us/step - loss: 0.0464 - acc: 0.9908 - val_loss: 0.1129 - val_acc: 0.9586
Epoch 8/10
18000/18000 [=====================

In [14]:
Right = 0
Wrong = 0
for i in range(10000):
    ind = np.random.randint(0, len(test_x))
    rowx, rowy = test_x[np.array([i])], test_y[np.array([i])]
    preds = model.predict_classes(rowx, verbose=0)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    
    #print('Q', q[::-1] if REVERSE else q, end=' ')
    #print('T', correct, end=' ')

    if correct == guess:
        Right += 1
    else:
        Wrong += 1
print(Right/10000)

0.7995
